# **Botnet Detection in IOT devices using an Autoencoder**

In [ ]:
!zip -r /content/file.zip /content/ae_model-DanminiDoorbell

  adding: content/ae_model-DanminiDoorbell/ (stored 0%)
  adding: content/ae_model-DanminiDoorbell/keras_metadata.pb (deflated 93%)
  adding: content/ae_model-DanminiDoorbell/saved_model.pb (deflated 90%)
  adding: content/ae_model-DanminiDoorbell/assets/ (stored 0%)
  adding: content/ae_model-DanminiDoorbell/variables/ (stored 0%)
  adding: content/ae_model-DanminiDoorbell/variables/variables.index (deflated 69%)
  adding: content/ae_model-DanminiDoorbell/variables/variables.data-00000-of-00001 (deflated 19%)


In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loading Required Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

## Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


## Autoencoder

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(115, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(37, activation="relu"),
            layers.Dense(28, activation="relu")
        ])
        self.decoder = Sequential([
            layers.Dense(37, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(115, activation="sigmoid")
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## Function To print Stats

In [ ]:
def print_stats(data, outcome):
    print(f"Shape of data: {data.shape}")
    print(f"Detected anomalies: {np.mean(outcome)*100}%")
    print()

## IOT Device 1: Danmini Doorbell

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("1.benign.csv")
X_train = benign[:40000]
# X_test0 = benign[40000:]
# X_test1 = load_nbaiot("1.mirai.scan.csv")
# X_test2 = load_nbaiot("1.mirai.ack.csv")
# X_test3 = load_nbaiot("1.mirai.syn.csv")
# X_test4 = load_nbaiot("1.mirai.udp.csv")
# X_test5 = load_nbaiot("1.mirai.udpplain.csv")
# X_test6 = load_nbaiot("1.gafgyt.combo.csv")
# X_test7 = load_nbaiot("1.gafgyt.junk.csv")
# X_test8 = load_nbaiot("1.gafgyt.scan.csv")
# X_test9 = load_nbaiot("1.gafgyt.tcp.csv")
# X_test10 = load_nbaiot("1.gafgyt.udp.csv")

In [ ]:
print(X_train.shape)

(40000, 115)


### Training the autoencoder

In [ ]:
checkpoint = ModelCheckpoint("ae_model", monitor='val_loss', verbose=1, save_best_only=True, mode='max')
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor, checkpoint]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)
print(threshold)

Epoch 1/800
865/875 [============================>.] - ETA: 0s - loss: 0.0029
Epoch 1: val_loss improved from -inf to 0.00139, saving model to ae_model
875/875 [==============================] - 7s 7ms/step - loss: 0.0029 - val_loss: 0.0014
Epoch 2/800
875/875 [==============================] - ETA: 0s - loss: 9.3745e-04
Epoch 2: val_loss did not improve from 0.00139
875/875 [==============================] - 4s 4ms/step - loss: 9.3745e-04 - val_loss: 0.0014
Epoch 3/800
864/875 [============================>.] - ETA: 0s - loss: 9.0842e-04
Epoch 3: val_loss did not improve from 0.00139
875/875 [==============================] - 4s 4ms/step - loss: 9.0574e-04 - val_loss: 0.0011
Epoch 4/800
875/875 [==============================] - ETA: 0s - loss: 8.6267e-04
Epoch 4: val_loss did not improve from 0.00139
875/875 [==============================] - 4s 4ms/step - loss: 8.6267e-04 - val_loss: 0.0011
Epoch 5/800
864/875 [============================>.] - ETA: 0s - loss: 7.6263e-04
Epoch 5: va

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (9548, 115)
Detected anomalies: 0.5492764339283829%

1
Shape of data: (107685, 115)
Detected anomalies: 100.0%

2
Shape of data: (102195, 115)
Detected anomalies: 100.0%

3
Shape of data: (122573, 115)
Detected anomalies: 100.0%

4
Shape of data: (237665, 115)
Detected anomalies: 100.0%

5
Shape of data: (81982, 115)
Detected anomalies: 100.0%

6
Shape of data: (59718, 115)
Detected anomalies: 100.0%

7
Shape of data: (29068, 115)
Detected anomalies: 100.0%

8
Shape of data: (29849, 115)
Detected anomalies: 100.0%

9
Shape of data: (92141, 115)
Detected anomalies: 100.0%

10
Shape of data: (105874, 115)
Detected anomalies: 100.0%



## IOT Device 2: Ecobee Thermostat

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("2.benign.csv")
X_train = benign[:10000]
X_test0 = benign[10000:]
X_test1 = load_nbaiot("2.mirai.scan.csv")
X_test2 = load_nbaiot("2.mirai.ack.csv")
X_test3 = load_nbaiot("2.mirai.syn.csv")
X_test4 = load_nbaiot("2.mirai.udp.csv")
X_test5 = load_nbaiot("2.mirai.udpplain.csv")
X_test6 = load_nbaiot("2.gafgyt.combo.csv")
X_test7 = load_nbaiot("2.gafgyt.junk.csv")
X_test8 = load_nbaiot("2.gafgyt.scan.csv")
X_test9 = load_nbaiot("2.gafgyt.tcp.csv")
X_test10 = load_nbaiot("2.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(40000, 115) (9548, 115) (107685, 115) (102195, 115) (122573, 115) (237665, 115) (81982, 115) (59718, 115) (29068, 115) (29849, 115) (92141, 115) (105874, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
875/875 [==============================] - 5s 5ms/step - loss: 0.0029 - val_loss: 0.0011
Epoch 2/800
875/875 [==============================] - 4s 5ms/step - loss: 8.7995e-04 - val_loss: 0.0023
Epoch 3/800
875/875 [==============================] - 4s 5ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 4/800
875/875 [==============================] - 5s 5ms/step - loss: 9.0517e-04 - val_loss: 0.0010
Epoch 5/800
875/875 [==============================] - 4s 5ms/step - loss: 7.9216e-04 - val_loss: 0.0011
Epoch 6/800
875/875 [==============================] - 4s 4ms/step - loss: 6.4929e-04 - val_loss: 7.0222e-04
Epoch 7/800
875/875 [==============================] - 4s 4ms/step - loss: 5.7037e-04 - val_loss: 5.8930e-04
Epoch 8/800
875/875 [==============================] - 4s 4ms/step - loss: 9.7086e-04 - val_loss: 0.0010
Epoch 9/800
875/875 [==============================] - 4s 5ms/step - loss: 7.5737e-04 - val_loss: 0.0011
Epoch 10/800
875/875 [==============================] -

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (9548, 115)
Detected anomalies: 0.5281504172388296%

1
Shape of data: (107685, 115)
Detected anomalies: 100.0%

2
Shape of data: (102195, 115)
Detected anomalies: 100.0%

3
Shape of data: (122573, 115)
Detected anomalies: 100.0%

4
Shape of data: (237665, 115)
Detected anomalies: 100.0%

5
Shape of data: (81982, 115)
Detected anomalies: 100.0%

6
Shape of data: (59718, 115)
Detected anomalies: 100.0%

7
Shape of data: (29068, 115)
Detected anomalies: 100.0%

8
Shape of data: (29849, 115)
Detected anomalies: 100.0%

9
Shape of data: (92141, 115)
Detected anomalies: 100.0%

10
Shape of data: (105874, 115)
Detected anomalies: 100.0%



## IOT Device 3: Ennio Doorbell

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("3.benign.csv")
X_train = benign[:30000]
X_test0 = benign[30000:]
X_test1 = load_nbaiot("3.gafgyt.combo.csv")
X_test2 = load_nbaiot("3.gafgyt.junk.csv")
X_test3 = load_nbaiot("3.gafgyt.scan.csv")
X_test4 = load_nbaiot("3.gafgyt.tcp.csv")
X_test5 = load_nbaiot("3.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape)

(30000, 115) (9100, 115) (53014, 115) (29797, 115) (28120, 115) (101536, 115) (103933, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)
encoder = load_model('ae_model')
# ae = Autoencoder()
# ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor, checkpoint]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
657/657 [==============================] - 4s 5ms/step - loss: 0.0147 - val_loss: 0.0069
Epoch 2/800
657/657 [==============================] - 3s 5ms/step - loss: 0.0143 - val_loss: 0.0064
Epoch 3/800
657/657 [==============================] - 3s 4ms/step - loss: 0.0143 - val_loss: 0.0064
Epoch 4/800
657/657 [==============================] - 3s 5ms/step - loss: 0.0143 - val_loss: 0.0057
Epoch 5/800
657/657 [==============================] - 3s 4ms/step - loss: 0.0143 - val_loss: 0.0065
Epoch 6/800
657/657 [==============================] - 6s 9ms/step - loss: 0.0143 - val_loss: 0.0062
Epoch 7/800
657/657 [==============================] - 3s 4ms/step - loss: 0.0143 - val_loss: 0.0062
Epoch 8/800
657/657 [==============================] - 3s 5ms/step - loss: 0.0143 - val_loss: 0.0059
Epoch 9/800
657/657 [==============================] - 4s 7ms/step - loss: 0.0143 - val_loss: 0.0062
Epoch 9: early stopping


### Testing the autoencoder

In [ ]:
# encoder = load_model('ae_model')
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, encoder(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (9100, 115)
Detected anomalies: 100.0%

1
Shape of data: (53014, 115)
Detected anomalies: 100.0%

2
Shape of data: (29797, 115)
Detected anomalies: 100.0%

3
Shape of data: (28120, 115)
Detected anomalies: 100.0%

4
Shape of data: (101536, 115)
Detected anomalies: 100.0%

5
Shape of data: (103933, 115)
Detected anomalies: 100.0%



## IOT Device 4: Philips B120N10 Baby Monitor

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("4.benign.csv")
X_train = benign[:160000]
X_test0 = benign[160000:]
X_test1 = load_nbaiot("4.mirai.scan.csv")
X_test2 = load_nbaiot("4.mirai.ack.csv")
X_test3 = load_nbaiot("4.mirai.syn.csv")
X_test4 = load_nbaiot("4.mirai.udp.csv")
X_test5 = load_nbaiot("4.mirai.udpplain.csv")
X_test6 = load_nbaiot("4.gafgyt.combo.csv")
X_test7 = load_nbaiot("4.gafgyt.junk.csv")
X_test8 = load_nbaiot("4.gafgyt.scan.csv")
X_test9 = load_nbaiot("4.gafgyt.tcp.csv")
X_test10 = load_nbaiot("4.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(160000, 115) (15240, 115) (103621, 115) (91123, 115) (118128, 115) (217034, 115) (80808, 115) (58152, 115) (28349, 115) (27859, 115) (92581, 115) (105782, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
3500/3500 [==============================] - 16s 4ms/step - loss: 0.0018 - val_loss: 0.0021
Epoch 2/800
3500/3500 [==============================] - 13s 4ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 3/800
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0076 - val_loss: 0.0105
Epoch 4/800
3500/3500 [==============================] - 15s 4ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 5/800
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 6/800
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0010 - val_loss: 9.9447e-04
Epoch 7/800
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 8/800
3500/3500 [==============================] - 15s 4ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 9/800
3500/3500 [==============================] - 15s 4ms/step - loss: 9.7155e-04 - val_loss: 0.0010
Epoch 10/800
3500/3500 [==============================] 

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (15240, 115)
Detected anomalies: 5.251006003034501%

1
Shape of data: (103621, 115)
Detected anomalies: 100.0%

2
Shape of data: (91123, 115)
Detected anomalies: 100.0%

3
Shape of data: (118128, 115)
Detected anomalies: 100.0%

4
Shape of data: (217034, 115)
Detected anomalies: 100.0%

5
Shape of data: (80808, 115)
Detected anomalies: 100.0%

6
Shape of data: (58152, 115)
Detected anomalies: 100.0%

7
Shape of data: (28349, 115)
Detected anomalies: 100.0%

8
Shape of data: (27859, 115)
Detected anomalies: 100.0%

9
Shape of data: (92581, 115)
Detected anomalies: 100.0%

10
Shape of data: (105782, 115)
Detected anomalies: 100.0%



## IOT Device 5: Provision PT 737E Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("5.benign.csv")
X_train = benign[:55000]
X_test0 = benign[55000:]
X_test1 = load_nbaiot("5.mirai.scan.csv")
X_test2 = load_nbaiot("5.mirai.ack.csv")
X_test3 = load_nbaiot("5.mirai.syn.csv")
X_test4 = load_nbaiot("5.mirai.udp.csv")
X_test5 = load_nbaiot("5.mirai.udpplain.csv")
X_test6 = load_nbaiot("5.gafgyt.combo.csv")
X_test7 = load_nbaiot("5.gafgyt.junk.csv")
X_test8 = load_nbaiot("5.gafgyt.scan.csv")
X_test9 = load_nbaiot("5.gafgyt.tcp.csv")
X_test10 = load_nbaiot("5.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(55000, 115) (7154, 115) (96781, 115) (60554, 115) (65746, 115) (156248, 115) (56681, 115) (61380, 115) (30898, 115) (29297, 115) (104510, 115) (104011, 115)


### Training the autoencoder

In [ ]:
checkpoint = ModelCheckpoint("security_camera_model", monitor='val_loss', verbose=1, save_best_only=True, mode='max')
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor, checkpoint]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
1192/1204 [============================>.] - ETA: 0s - loss: 0.0035
Epoch 1: val_loss improved from -inf to 0.00203, saving model to security_camera_model
1204/1204 [==============================] - 8s 6ms/step - loss: 0.0035 - val_loss: 0.0020
Epoch 2/800
1192/1204 [============================>.] - ETA: 0s - loss: 0.0012
Epoch 2: val_loss did not improve from 0.00203
1204/1204 [==============================] - 3s 3ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 3/800
1204/1204 [==============================] - ETA: 0s - loss: 9.5109e-04
Epoch 3: val_loss improved from 0.00203 to 0.00479, saving model to security_camera_model
1204/1204 [==============================] - 5s 4ms/step - loss: 9.5109e-04 - val_loss: 0.0048
Epoch 4/800
1193/1204 [============================>.] - ETA: 0s - loss: 0.0011
Epoch 4: val_loss did not improve from 0.00479
1204/1204 [==============================] - 3s 3ms/step - loss: 0.0011 - val_loss: 8.1781e-04
Epoch 5/800
1192/1204 [==========

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (7154, 115)
Detected anomalies: 2.403506291531175%

1
Shape of data: (96781, 115)
Detected anomalies: 100.0%

2
Shape of data: (60554, 115)
Detected anomalies: 100.0%

3
Shape of data: (65746, 115)
Detected anomalies: 100.0%

4
Shape of data: (156248, 115)
Detected anomalies: 100.0%

5
Shape of data: (56681, 115)
Detected anomalies: 100.0%

6
Shape of data: (61380, 115)
Detected anomalies: 100.0%

7
Shape of data: (30898, 115)
Detected anomalies: 100.0%

8
Shape of data: (29297, 115)
Detected anomalies: 100.0%

9
Shape of data: (104510, 115)
Detected anomalies: 100.0%

10
Shape of data: (104011, 115)
Detected anomalies: 100.0%



## IOT Device 6: Provision PT 838 Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("6.benign.csv")
X_train = benign[:10000]
X_test0 = benign[10000:]
X_test1 = load_nbaiot("6.mirai.scan.csv")
X_test2 = load_nbaiot("6.mirai.ack.csv")
X_test3 = load_nbaiot("6.mirai.syn.csv")
X_test4 = load_nbaiot("6.mirai.udp.csv")
X_test5 = load_nbaiot("6.mirai.udpplain.csv")
X_test6 = load_nbaiot("6.gafgyt.combo.csv")
X_test7 = load_nbaiot("6.gafgyt.junk.csv")
X_test8 = load_nbaiot("6.gafgyt.scan.csv")
X_test9 = load_nbaiot("6.gafgyt.tcp.csv")
X_test10 = load_nbaiot("6.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(10000, 115) (88514, 115) (97096, 115) (57997, 115) (61851, 115) (158608, 115) (53785, 115) (57530, 115) (29068, 115) (28397, 115) (89387, 115) (104658, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)
encoder =  load_model('security_camera_model')

# ae = Autoencoder()
# ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
encoder.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 2/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0028 - val_loss: 0.0016
Epoch 3/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0018 - val_loss: 0.0013
Epoch 4/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 5/800
219/219 [==============================] - 1s 2ms/step - loss: 0.0015 - val_loss: 9.7908e-04
Epoch 6/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0015 - val_loss: 0.0023
Epoch 7/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0032 - val_loss: 0.0019
Epoch 8/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0026 - val_loss: 0.0027
Epoch 9/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0027 - val_loss: 0.0018
Epoch 10/800
219/219 [==============================] - 1s 3ms/step - loss: 0.0019 - va

### Testing the autoencoder

In [ ]:

def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, encoder(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (88514, 115)
Detected anomalies: 7.30044214263906%

1
Shape of data: (97096, 115)
Detected anomalies: 100.0%

2
Shape of data: (57997, 115)
Detected anomalies: 100.0%

3
Shape of data: (61851, 115)
Detected anomalies: 100.0%

4
Shape of data: (158608, 115)
Detected anomalies: 100.0%

5
Shape of data: (53785, 115)
Detected anomalies: 100.0%

6
Shape of data: (57530, 115)
Detected anomalies: 100.0%

7
Shape of data: (29068, 115)
Detected anomalies: 100.0%

8
Shape of data: (28397, 115)
Detected anomalies: 100.0%

9
Shape of data: (89387, 115)
Detected anomalies: 0.0%

10
Shape of data: (104658, 115)
Detected anomalies: 0.0%



## IOT Device 7: Samsung SNH 1011 N Webcam

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("7.benign.csv")
X_train = benign[:50000]
X_test0 = benign[50000:]
X_test1 = load_nbaiot("7.gafgyt.combo.csv")
X_test2 = load_nbaiot("7.gafgyt.junk.csv")
X_test3 = load_nbaiot("7.gafgyt.scan.csv")
X_test4 = load_nbaiot("7.gafgyt.tcp.csv")
X_test5 = load_nbaiot("7.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape)

(50000, 115) (2150, 115) (58669, 115) (28305, 115) (27698, 115) (97783, 115) (110617, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
1094/1094 [==============================] - 7s 6ms/step - loss: 0.0030 - val_loss: 5.5601e-04
Epoch 2/800
1094/1094 [==============================] - 6s 5ms/step - loss: 0.0019 - val_loss: 0.0012
Epoch 3/800
1094/1094 [==============================] - 5s 5ms/step - loss: 0.0017 - val_loss: 6.0940e-04
Epoch 4/800
1094/1094 [==============================] - 6s 6ms/step - loss: 0.0012 - val_loss: 6.7967e-04
Epoch 5/800
1094/1094 [==============================] - 5s 4ms/step - loss: 0.0013 - val_loss: 5.9837e-04
Epoch 6/800
1094/1094 [==============================] - 5s 5ms/step - loss: 0.0014 - val_loss: 3.7756e-04
Epoch 7/800
1094/1094 [==============================] - 6s 6ms/step - loss: 0.0013 - val_loss: 3.8458e-04
Epoch 8/800
1094/1094 [==============================] - 5s 5ms/step - loss: 0.0012 - val_loss: 4.0895e-04
Epoch 9/800
1094/1094 [==============================] - 6s 5ms/step - loss: 0.0012 - val_loss: 5.0406e-04
Epoch 10/800
1094/1094 [=================

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (2150, 115)
Detected anomalies: 21.26631222812953%

1
Shape of data: (58669, 115)
Detected anomalies: 100.0%

2
Shape of data: (28305, 115)
Detected anomalies: 100.0%

3
Shape of data: (27698, 115)
Detected anomalies: 100.0%

4
Shape of data: (97783, 115)
Detected anomalies: 100.0%

5
Shape of data: (110617, 115)
Detected anomalies: 100.0%



## IOT Device 8: SimpleHome XCS7 1002 WHT Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("8.benign.csv")
X_train = benign[:40000]
X_test0 = benign[40000:]
X_test1 = load_nbaiot("8.mirai.scan.csv")
X_test2 = load_nbaiot("8.mirai.ack.csv")
X_test3 = load_nbaiot("8.mirai.syn.csv")
X_test4 = load_nbaiot("8.mirai.udp.csv")
X_test5 = load_nbaiot("8.mirai.udpplain.csv")
X_test6 = load_nbaiot("8.gafgyt.combo.csv")
X_test7 = load_nbaiot("8.gafgyt.junk.csv")
X_test8 = load_nbaiot("8.gafgyt.scan.csv")
X_test9 = load_nbaiot("8.gafgyt.tcp.csv")
X_test10 = load_nbaiot("8.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(40000, 115) (6585, 115) (45930, 115) (111480, 115) (125715, 115) (151879, 115) (78244, 115) (54283, 115) (28579, 115) (27825, 115) (88816, 115) (103720, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

encoder = load_model("security_camera_model");
# ae = Autoencoder()
# ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
encoder.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
875/875 [==============================] - 3s 3ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 2/800
875/875 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 8.8752e-04
Epoch 3/800
875/875 [==============================] - 3s 3ms/step - loss: 0.0013 - val_loss: 0.0015
Epoch 4/800
875/875 [==============================] - 3s 3ms/step - loss: 0.0013 - val_loss: 0.0017
Epoch 5/800
875/875 [==============================] - 3s 3ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 6/800
875/875 [==============================] - 5s 6ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 7/800
875/875 [==============================] - 4s 4ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 7: early stopping


### Testing the autoencoder

In [ ]:

def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, encoder(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (6585, 115)
Detected anomalies: 0.0%

1
Shape of data: (45930, 115)
Detected anomalies: 97.62699295513534%

2
Shape of data: (111480, 115)
Detected anomalies: 100.0%

3
Shape of data: (125715, 115)
Detected anomalies: 100.0%

4
Shape of data: (151879, 115)
Detected anomalies: 100.0%

5
Shape of data: (78244, 115)
Detected anomalies: 100.0%

6
Shape of data: (54283, 115)
Detected anomalies: 100.0%

7
Shape of data: (28579, 115)
Detected anomalies: 100.0%

8
Shape of data: (27825, 115)
Detected anomalies: 100.0%

9
Shape of data: (88816, 115)
Detected anomalies: 0.0%

10
Shape of data: (103720, 115)
Detected anomalies: 0.0%



## IOT Device 9: SimpleHome XCS7 1003 WHT Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("9.benign.csv")
X_train = benign[:15000]
X_test0 = benign[15000:]
X_test1 = load_nbaiot("9.mirai.scan.csv")
X_test2 = load_nbaiot("9.mirai.ack.csv")
X_test3 = load_nbaiot("9.mirai.syn.csv")
X_test4 = load_nbaiot("9.mirai.udp.csv")
X_test5 = load_nbaiot("9.mirai.udpplain.csv")
X_test6 = load_nbaiot("9.gafgyt.combo.csv")
X_test7 = load_nbaiot("9.gafgyt.junk.csv")
X_test8 = load_nbaiot("9.gafgyt.scan.csv")
X_test9 = load_nbaiot("9.gafgyt.tcp.csv")
X_test10 = load_nbaiot("9.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(15000, 115) (4528, 115) (43674, 115) (107187, 115) (122479, 115) (157084, 115) (84436, 115) (59398, 115) (27413, 115) (28572, 115) (98075, 115) (102980, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/800
329/329 [==============================] - 3s 5ms/step - loss: 0.0061 - val_loss: 0.0016
Epoch 2/800
329/329 [==============================] - 2s 5ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 3/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0019 - val_loss: 0.0013
Epoch 4/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 5/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0036 - val_loss: 0.0017
Epoch 6/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0017 - val_loss: 0.0013
Epoch 7/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 8/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0013 - val_loss: 8.8949e-04
Epoch 9/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0020 - val_loss: 0.0015
Epoch 10/800
329/329 [==============================] - 1s 4ms/step - loss: 0.0016 - va

### Testing the autoencoder

In [ ]:
def predict(x, threshold=threshold, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (4528, 115)
Detected anomalies: 0.0%

1
Shape of data: (43674, 115)
Detected anomalies: 100.0%

2
Shape of data: (107187, 115)
Detected anomalies: 100.0%

3
Shape of data: (122479, 115)
Detected anomalies: 100.0%

4
Shape of data: (157084, 115)
Detected anomalies: 100.0%

5
Shape of data: (84436, 115)
Detected anomalies: 100.0%

6
Shape of data: (59398, 115)
Detected anomalies: 100.0%

7
Shape of data: (27413, 115)
Detected anomalies: 100.0%

8
Shape of data: (28572, 115)
Detected anomalies: 100.0%

9
Shape of data: (98075, 115)
Detected anomalies: 0.0%

10
Shape of data: (102980, 115)
Detected anomalies: 0.0%

